In [19]:
# Generate inital dataframe
import pandas as pd
from faker import Faker
import random

fake = Faker()
num_rows = 10
ids = [fake.uuid4() for _ in range(num_rows)]
dates = [fake.date_between(start_date='-1y', end_date='-1m').strftime('%Y-%m-%d') for _ in range(num_rows)]
metric_values = [round(random.uniform(0, 10),2) for _ in range(num_rows)]
dataframe = pd.DataFrame({'unique_key': ids, 'date': dates, 'metric_value': metric_values})

# Insert duplication
n = 2  # Number of rows to duplicate
indices = random.sample(range(len(dataframe)), n)
subset = dataframe.iloc[indices]

# Append the selected rows to the DataFrame
dataframe = pd.concat([dataframe, subset])
len(dataframe)

12

In [20]:

def rename_duplicates(df):
    """
    Rename duplicate 'unique_key' values in the DataFrame.

    Parameters:
    - df: pandas DataFrame with a 'unique_key' column.

    Returns:
    - DataFrame with renamed duplicates.
    """
    
    # Find duplicated rows based on the 'unique_key' column
    duplicates = df['unique_key'].duplicated(keep="first")
    
    # Create a series with the same index as the dataframe for counting duplicates
    counter = df[duplicates].groupby('unique_key').cumcount() + 1
    
    # Format the 'unique_key' for duplicates
    df.loc[duplicates, 'unique_key'] = df['unique_key'][duplicates] + "-duplicate-" + counter.astype(str)

    return df

renamed_dataframe = rename_duplicates(dataframe)
print(renamed_dataframe)

                                         unique_key        date  metric_value
0              49d54cca-e2a9-4148-92be-2a6f208c92a1  2023-07-23          8.26
1              f5cf8830-ff3e-4eed-b0b1-c082b575aadf  2023-01-26          3.64
2              aeebfe39-dc7c-42d4-91d0-4fa952624b98  2023-05-11          7.45
3              d04522fa-0196-455b-9a13-694abc21dfda  2023-02-20          6.81
4              9bbff6f3-0b8d-43db-821b-d8672096929b  2023-05-24          1.62
5              1b74b148-fb2b-47f6-abc1-ffac54633431  2022-10-18          1.57
6              90799d8b-1ef1-45b7-8796-2f4ebb3dec01  2023-06-15          5.40
7              042e489b-eb6a-4d1d-93f5-0e536799e2d6  2023-10-02          8.28
8              364e4b00-2094-4435-a408-e7997cbcff9c  2023-07-03          5.11
9              973e87c5-6873-4b46-b948-9d3a7b893108  2022-10-18          0.12
2  aeebfe39-dc7c-42d4-91d0-4fa952624b98-duplicate-1  2023-05-11          7.45
3  d04522fa-0196-455b-9a13-694abc21dfda-duplicate-1  2023-02-20 

In [8]:
from dotenv import load_dotenv
from github import Github
import os
import sys
load_dotenv("../.env")
gh_token = os.getenv("GH_TOKEN")
gh_client = Github(gh_token, timeout=60)
repo = os.getenv("REPO") or "gh_org/repo"
file_path = repo+"/duplicates/test.csv"

In [25]:

sys.path.append('..')
import importlib
import datagit.github_connector
importlib.reload(datagit.github_connector)
from datagit.github_connector import store_metric



store_metric(ghClient=gh_client, dataframe=dataframe, filepath=file_path, branch="test")

Storing metric...
Metric not found, creating it on branch: main
Commit: New data: duplicates/test.csv
Metric stored
